In [9]:
import pandas as pd
import numpy as np
from transformers import pipeline

In [2]:
data = pd.read_csv('books_categorized.csv')

In [3]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [18]:
emotion_labels = ['anger','disgust','fear','joy','neutral','sadness','surprise']
emotion_scores = {labels:[] for labels in emotion_labels}
isbn = []

def calculate_emotion_scores(predictions):
    each_emotion_score = {label: [] for label in emotion_labels}

    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x['label'])
        for index, label in enumerate(emotion_labels):
            each_emotion_score[label].append(sorted_predictions[index]['score'])
    return {label: np.max(scores) if scores else 0 for label, scores in each_emotion_score.items()}

In [19]:
from tqdm import tqdm

In [20]:
for i in tqdm(range(len(data))):
  isbn.append(data['isbn13'][i])
  sentences = data['description'][i].split(".")
  predictions = classifier(sentences)
  max_scores = calculate_emotion_scores(predictions)
  for label in emotion_labels:
    emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [02:53<00:00, 30.04it/s]


In [21]:
book_sentiments = pd.DataFrame(emotion_scores)
book_sentiments['isbn13'] = isbn

In [24]:
book_sentiments.head()

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273591,0.928168,0.932797,0.646216,0.967158,0.729603,9780002005883
1,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766,9780006178736
3,0.351484,0.150723,0.360706,0.251881,0.732685,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078766,9780006280934


In [25]:
book_sentiments_analyzed = pd.merge(data, book_sentiments, on='isbn13')

In [27]:
book_sentiments_analyzed.to_csv('books_sentiments.csv',index=False)